In [2]:
import torch
import cv2
import matplotlib.pyplot as plt
import numpy as np
from glob import glob
import os
from datetime import datetime
import time
import random as rd 
import pandas as pd
from duetwebapi import DuetWebAPI as DWA


<div class="alert alert-info"> <h1>Use Case: Echtzeitanpassungen an 3D-gedruckten Bauteilen </h1>

1. Eingabe: Bilder von Webcam, vorverarbeitet für das Modell
2. Ausgabe: Auswertung der Bilder; G-Code Modifikationen, resultierend aus den Ergebnissen

In [2]:
printer_hostname = '192.0.2.1'
printer = DWA(f'http://{printer_hostname}')


<div class="alert alert-info"><h2> Modell Laden mit Pytorch-Hub

In [4]:
model = torch.hub.load('yolov5', 'custom', path='model.onnx', source='local', force_reload=True, verbose=False)


YOLOv5  2024-5-29 Python-3.9.18 torch-2.3.1+cu118 CUDA:0 (NVIDIA GeForce GTX 970, 4096MiB)

Loading model.onnx for ONNX Runtime inference...
Adding AutoShape... 


<div class="alert alert-info"><h2> Funktionen zur Bildaufnahme und Preprocessing

In [4]:
def bash_command():
    file = 'image_cap.png'
    befehl = f'fswebcam -r 640x480 -d /dev/video2 --no-banner {file}'
    os.system(befehl)
    print("foto aufgenommen.")
    return file

In [5]:
def resize_with_padding(img_pil, target_size):
    h, w = img_pil.shape[:2]
    scale = target_size / max(h, w)
    new_w = int(w * scale)
    new_h = int(h * scale)

    resized_img = cv2.resize(img_pil, (new_w, new_h), interpolation=cv2.INTER_LINEAR)

    padded_img = np.zeros((target_size, target_size, 3), dtype=np.uint8)

    top = (target_size - new_h) // 2
    left = (target_size - new_w) // 2
    padded_img[top:top + new_h, left:left + new_w] = resized_img

    return padded_img



<div class="alert alert-success">
  <h3>Druckerinformationen</h3>
  <ul>
    <li><strong>tool</strong>: Aktives Werkzeug</li>
    <li><strong>efactor</strong>: Extrusionsfaktor in %</li>
    <li><strong>sfactor</strong>: Geschwindigkeitsfaktor in %</li>
    <li><strong>heaters</strong>: Temperaturen; 3er Array als [T0, T1, Bett]</li>
    <li><strong>babystep</strong>: Abstand Nozzle &lt;-&gt; Bett</li>
    <li><strong>hstat</strong>: Status der Heizer; 3er Array mit 0 = aus, 1 = an, 2 = standby</li>
  </ul>
<par> speichere als Variablen </par>
</div>


<div class="alert alert-success"><h1> Start

In [177]:
import json

# Originales Dictionary
status = printer.send_code('M408')
# Extrahiere den String im Schlüssel 'response'
response_str = status['response']

# Wandle den String in ein echtes Dictionary um
data = json.loads(response_str)

tool = int(data['tool'])
efactor = data['efactor'][tool] #list -> float
sfactor = data['sfactor'] # float
temp = data['active'][tool+1] 
babystep = data['babystep']


print(f'tool:\t\t\t{tool}\nextrusions:\t\t{efactor}\nspeed:\t\t\t{sfactor}%\ntemp:\t\t\t{temp}°C\n')


tool:			1
extrusions:		125.0
speed:			90.0%
temp:			210.0°C



<div class="alert alert-info"> <h3> Bildaufnahme (raw)

In [84]:
im = bash_command()

foto aufgenommen.


--- Opening /dev/video2...
Trying source module v4l2...
/dev/video2 opened.
No input was specified, using the first.
Error selecting input 0
VIDIOC_S_INPUT: Device or resource busy


<div class="alert alert-info"><h3> Bild einlesen und Modell anwenden

In [178]:
#im = 'testbilder/my_photo-15.jpg'
path = cv2.imread(im)
img = resize_with_padding(path, 640)

img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
#cv2.imwrite('testing_image.jpg', img)
result = model(img)
result.pandas().xyxy[0]
cv2.imwrite('testbilder/results/detection2.jpg', np.squeeze(result.render()))

True

<div class="alert alert-info">
<h2>Modell auf alle Bilder im Verzeichnis anwenden

In [10]:
from glob import glob
files = glob('test_images/*.jpg')

for i, file in enumerate(files):
    im = cv2.imread(file)
    im = resize_with_padding(im, 640)
    im = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
    result = model(im)
    result.pandas().xyxy[0]
    cv2.imwrite(f'Detections/detection_{i}.jpg', np.squeeze(result.render()))

'test_images\\anotherstringy.jpg'

<div class="alert alert-info">
<h3>Weiterverwenden der Ergebnisse </h3>


</div>

In [179]:
labels = result.pandas().xyxy[0]['name']
conf = result.pandas().xyxy[0]['confidence']
for i in range(len(labels)):
    print(f'{labels[i]}\t->\t{round(conf[i], 2)}')


stringing	->	0.43


<div class="alert alert-info">
<h3>Filtere die Ergebnisse </h3>
nur Detections über einem <b>bestimmten Schwellwert</b> sollen berücksichtigt werden für die G-Code-Anpassungen

</div>

In [180]:
confidence_threshold = 0.369
filtered = result.pandas().xyxy[0][result.pandas().xyxy[0]['confidence'] > confidence_threshold]['name'].tolist()

<div class="alert alert-info">
<h3>Verarbeite die Informationen aus Drucker-Status </h3>


</div>

In [181]:
sfactor, efactor, temp

(90.0, 125.0, 210.0)

In [182]:
def adjust_code(filtered_classes, sfactor, efactor, temp):
    # Überprüfe auf Stringing und passe den Geschwindigkeitsfaktor (sfactor) an
    with open('output.log', 'a') as log:
        if 'stringing' in filtered_classes:
            print('Stringing erkannt! Passe die Parameter an.')

            if temp > 195:
                temp -= 10
                print(f'Extrusionstemperatur auf {temp}°C reduziert')
            else:
                print(f'Temperatur auf Minimum reduziert! ({temp}°C)')    

            if sfactor > 80.0:
                sfactor -= 5
                print(f'Geschwindigkeitsfaktor auf {sfactor}% reduziert.')
                
            else:
                print(f'Speedfaktor auf ein Minimum reduziert! ({sfactor}%)')

            #if efactor > 95.0:
             #   efactor -= 5
              #  print(f'ExFac auf {efactor}% reduziert')
        # Überprüfe auf Overextrusion und passe den Extrusionsfaktor (efactor) an
        if 'overextrusion' in filtered_classes:
            print('Overextrusion erkannt! Passe die Parameter an.')
            if efactor > 95.0:
                efactor -= 5
                print(f'Extrusionsfaktor auf {efactor}% reduziert.')
                
            else:
                print(f'Extrusionsfaktor auf Minimum! ({efactor}%)')

    # Rückgabe der aktualisierten Werte
    return sfactor, efactor, temp
sf, ef, t = adjust_code(filtered, sfactor, efactor, temp)


Stringing erkannt! Passe die Parameter an.
Extrusionstemperatur auf 200.0°C reduziert
Geschwindigkeitsfaktor auf 85.0% reduziert.


In [183]:
sf, ef, t


(85.0, 125.0, 200.0)

<div class="alert alert-success"><h2>Sende aktualisierte Parameter an Drucker-API

In [184]:
printer.send_code(f'M221 S{ef}')
printer.send_code(f'M104 S{t}')
printer.send_code(f'M220 S{sf}')

{'response': '\n\n\n'}

<div class="alert alert-info"> <h2> optional: Schreibe Verlauf in Logfile

In [185]:
import logging
logging.basicConfig(filename='output.log', level=logging.INFO)
with open('output.log', 'a') as log:
    if 'stringing' in labels:
        log.write(f'{datetime.now().time().strftime("%H:%M:%S")}\tStringing; params:\t\t speedfactor={sf}, exfactor={ef}, temp={t}\n')
    elif 'overextrusion' in labels:
        log.write(f'{datetime.now().time().strftime("%H:%M:%S")}\toverextrusion; params:\t\t speedfactor={sf}, exfactor={ef}, temp={t}\n')
    else: 
        log.write(f'{datetime.now().time().strftime("%H:%M:%S")}\tpass\n')    